In [1]:
import pyscf
import numpy
from pyscf import gto, scf, ci,df,lib, dft
import scipy
from time import ctime, time
import numgrid

In [7]:
import pyscf
import numpy
from pyscf import gto, scf, ci,df,lib, dft
import scipy
from time import ctime, time
import numgrid
lib.num_threads(1)
TimeStart = time()
mol = gto.Mole(                                                                                               
        atom='''
         H       0.0000 0.0000 0.0000                 
         H       0.7600 0.0000 0.0000''',
        charge=-1,spin=1,basis='cc-pVQZ',verbose=4
    ).build()
#method = scf.UHF(mol).density_fit(auxbasis="def2-tzvp-jkfit")
method = dft.UKS(mol)
method.scf
method.xc = 'x3lypg'
method.init_guess = '1e'
method.grids.becke_scheme = dft.original_becke
method.grids.level = 3
method.kernel()
print("Total job time: %10.2f(wall)" %(time()-TimeStart))

System: uname_result(system='Linux', node='igor-Latitude-5591', release='5.4.0-122-generic', version='#138-Ubuntu SMP Wed Jun 22 15:00:31 UTC 2022', machine='x86_64', processor='x86_64')  Threads 1
Python 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
numpy 1.16.4  scipy 1.2.1
Date: Fri Aug 12 00:29:11 2022
PySCF version 2.0.1
PySCF path  /home/igor/Data/opt/anaconda3/lib/python3.6/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 3
[INPUT] charge = -1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT]  2 H      0.760000000000   0.000000000000   0.000000000000 AA    1.436191854669   0.000000000000   0.000000000000 Bohr

nuclear repulsion = 0.696285803842105
number of shells = 20
number of NR pGTOs = 64
number of 

In [57]:
# get density matrix from '1e'
dm = method.init_guess_by_1e()
print("dm_a")
for x in dm[0]:
    print("%10.6f"*len(x) %tuple([y for y in x]))
print("dm_b")
for x in dm[1]:
    print("%10.6f"*len(x) %tuple([y for y in x]))
# density based on a given grids setting
#print(coords.shape)
# ao_value = pyscf.dft.numint.eval_ao(mol,coords)
# rho_value = pyscf.dft.numint.eval_rho(mol,ao_value,dm)
ni = dft.numint.NumInt()
nelec, exc, vxc = ni.nr_uks(mol, method.grids, 'pbe,pbe', dm)
print(nelec, exc)
for x in vxc:
    print("16.8f"*10 %tuple([y for y in x]))
#     print("%10.6f"*2 %tuple([y for y in x]))
# result = pyscf.dft.rks.get_veff(method, mol, dm)
# print(result.ecoul.real,result.exc.real)

# print(result[0])

# print(numpy.einsum('ij,ji->',method.get_hcore(),dm[1]))

# print(method.scf_summary, method.scf_summary['e1']+method.scf_summary['coul']+method.scf_summary['exc']+method.scf_summary['nuc'])

Initial guess from hcore.
  alpha nocc = 2  HOMO = -0.611211671131453  LUMO = -0.315915792325708
  beta  nocc = 1  HOMO = -1.26716325458686  LUMO = -0.611211671131453

WARN: system HOMO -0.611211671131453 >= system LUMO -0.611211671131453

dm_a
  0.528715  0.617341 -0.019663 -0.000000  0.000000 -0.028103 -0.557289 -0.067642  0.000000 -0.000000
  0.617341  1.242565 -0.089208 -0.000000  0.000000 -0.557289 -1.235361 -0.094964  0.000000 -0.000000
 -0.019663 -0.089208  0.009143  0.000000 -0.000000  0.067642  0.094964  0.004545 -0.000000  0.000000
 -0.000000 -0.000000  0.000000  0.000000 -0.000000  0.000000  0.000000  0.000000 -0.000000  0.000000
  0.000000  0.000000 -0.000000 -0.000000  0.000000 -0.000000 -0.000000 -0.000000  0.000000 -0.000000
 -0.028103 -0.557289  0.067642  0.000000 -0.000000  0.528715  0.617341  0.019663 -0.000000  0.000000
 -0.557289 -1.235361  0.094964  0.000000 -0.000000  0.617341  1.242565  0.089208 -0.000000  0.000000
 -0.067642 -0.094964  0.004545  0.000000 -0.0000

TypeError: not all arguments converted during string formatting

In [23]:
dm = method.init_guess_by_1e()
coords = numpy.asarray(method.grids.coords)
ni = dft.numint.NumInt()
ao_value = pyscf.dft.numint.eval_ao(mol,coords,deriv=1)
rho_value = pyscf.dft.numint.eval_rho(mol,ao_value,dm,xctype='GGA')
nelec, exc, vxc = ni.nr_rks(mol, method.grids, 'pbe,pbe', dm)
print(nelec, exc)
for x in vxc:
    print("%10.6f"*len(x) %tuple(x))
# exc, vxc = ni.eval_xc('pbe,pbe',rho_value,spin=0,relativity=0,deriv=1,omega=None)[:2]
# print("exc:")
# for x in exc[:100]:
#     print(x)
# vxc = numpy.asarray(vxc)
# print(vxc.shape)
# print("vxc:")
# for x in range(100):
#     print("%16.8f,%16.8f" %(vxc[0][x],vxc[1][x]))

Initial guess from hcore.
  HOMO = -1.26716325458686  LUMO = -0.611211671131453
1.9999999943069717 -0.7935986663251213
 -0.508667 -0.272814 -0.089796 -0.000000  0.000000 -0.294695 -0.233581  0.314248 -0.000000 -0.000000
 -0.272814 -0.236010 -0.067739 -0.000000  0.000000 -0.233581 -0.219023  0.117793 -0.000000  0.000000
 -0.089796 -0.067739 -0.458197 -0.000000  0.000000 -0.314248 -0.117793  0.171290  0.000000 -0.000000
 -0.000000 -0.000000 -0.000000 -0.410615  0.000000 -0.000000  0.000000  0.000000 -0.214558 -0.000000
  0.000000  0.000000  0.000000  0.000000 -0.410615  0.000000  0.000000  0.000000 -0.000000 -0.214558
 -0.294695 -0.233581 -0.314248 -0.000000  0.000000 -0.508667 -0.272814  0.089796  0.000000 -0.000000
 -0.233581 -0.219023 -0.117793  0.000000  0.000000 -0.272814 -0.236010  0.067739  0.000000 -0.000000
  0.314248  0.117793  0.171290  0.000000  0.000000  0.089796  0.067739 -0.458197  0.000000 -0.000000
 -0.000000 -0.000000  0.000000 -0.214558 -0.000000  0.000000  0.000000  0

In [54]:
# for spin polarization
dm = method.init_guess_by_1e()
coords = numpy.asarray(method.grids.coords)
ni = dft.numint.NumInt()
ao_value = pyscf.dft.numint.eval_ao(mol,coords,deriv=1)
rho_value_a = pyscf.dft.numint.eval_rho(mol,ao_value,dm[0],xctype='GGA')
rho_value_b = pyscf.dft.numint.eval_rho(mol,ao_value,dm[1],xctype='GGA')
exc, vxc = ni.eval_xc('pbe,pbe',(rho_value_a,rho_value_b),spin=1,relativity=0,deriv=1,omega=None)[:2]
# print(rho_value_b.shape)
# print("exc:")
# print(exc.shape)
# for x in range(100):
#     print("%16.8f,%16.8f,%16.8f" %(exc[x],rho_value_a[x],rho_value_b[x]))

# print("%16.8f,%16.8f,%16.8f" %(exc[17950-1],rho_value_a[17950-1],rho_value_b[17950-1]))
# exc_total = [0.0,0.0]
# for x in range(len(method.grids.weights)):
#     exc_total[0] += exc[x]*rho_value_a[x]*method.grids.weights[x]
#     exc_total[1] += exc[x]*rho_value_b[x]*method.grids.weights[x]
# print(exc_total,sum(exc_total))
vxc = numpy.asarray(vxc)
print(vxc.shape)
print("vxc:")
# print(vxc[1])
for x in range(100):
    print("%16.8f,%16.8f,%16.8f" %(vxc[1][x][0],vxc[1][x][1],vxc[1][x][2]))

Initial guess from hcore.
  alpha nocc = 2  HOMO = -0.611211671131453  LUMO = -0.315915792325708
  beta  nocc = 1  HOMO = -1.26716325458686  LUMO = -0.611211671131453

WARN: system HOMO -0.611211671131453 >= system LUMO -0.611211671131453

(4,)
vxc:
     -0.00411880,      0.01628700,     -0.01609553
     -0.00411004,      0.01630468,     -0.01606670
     -0.00409737,      0.01633203,     -0.01601970
     -0.00408402,      0.01636623,     -0.01595406
     -0.00407830,      0.01639786,     -0.01587505
     -0.00409613,      0.01640780,     -0.01579651
     -0.00416329,      0.01636532,     -0.01574358
     -0.00431557,      0.01623102,     -0.01575335
     -0.00459431,      0.01597047,     -0.01587071
     -0.00503562,      0.01558210,     -0.01613774
     -0.00413202,      0.01626148,     -0.01613558
     -0.00414322,      0.01624063,     -0.01616724
     -0.00416506,      0.01620138,     -0.01622492
     -0.00420502,      0.01613273,     -0.01632152
     -0.00427527,      0.01601781,  

In [49]:
# weight = numpy.asarray(method.grids.weights)
weight = numpy.array([1.0]*len(method.grids.weights))
wv = pyscf.dft.numint._rks_gga_wv0(rho_value, vxc,weight)
# print(wv.shape)
# print("wv:")
# for x in range(100):
#     print("%16.8f,%16.8f,%16.8f,%16.8f" %(wv[0][x],wv[1][x],wv[2][x],wv[3][x]))
awo=None
awo = pyscf.dft.numint._scale_ao(ao_value, wv, out=awo)
print(awo.shape)
print("awo:")
for x in range(100):
    print("%16.8f,%16.8f" %(awo[x][0]*2,awo[x][1]*2))

(17042, 2)
awo:
     -0.48122874,     -0.10214704
     -0.48132753,     -0.10236940
     -0.48148256,     -0.10275447
     -0.48168288,     -0.10335197
     -0.48188565,     -0.10421358
     -0.48199856,     -0.10539240
     -0.48185850,     -0.10694229
     -0.48120734,     -0.10891677
     -0.47966732,     -0.11136703
     -0.47672126,     -0.11433883
     -0.48108749,     -0.10185260
     -0.48097297,     -0.10163029
     -0.48075869,     -0.10124545
     -0.48038616,     -0.10064859
     -0.47976342,     -0.09978853
     -0.47874823,     -0.09861296
     -0.47712788,     -0.09706907
     -0.47459691,     -0.09510437
     -0.47073565,     -0.09266767
     -0.46499531,     -0.08971029
     -0.48115806,     -0.10199966
     -0.48114991,     -0.10199883
     -0.48111920,     -0.10199572
     -0.48102996,     -0.10198668
     -0.48081230,     -0.10196462
     -0.48034469,     -0.10191714
     -0.47943232,     -0.10182419
     -0.47778299,     -0.10165512
     -0.47498314,     -0.1013651

In [47]:
ao_value = pyscf.dft.numint.eval_ao(mol,coords,deriv=1)
print(ao_value.shape)
for i in range(ao_value.shape[1]):
    print("%16.8f,%16.8f,%6i" %(ao_value[1][i][0],ao_value[1][i][1],i))

(4, 17308, 10)
     -0.00602455,     -0.00003845,     0
     -0.01512194,     -0.00009652,     1
     -0.03086721,     -0.00019705,     2
     -0.05526326,     -0.00035304,     3
     -0.09031826,     -0.00057792,     4
     -0.13791345,     -0.00088560,     5
     -0.19954936,     -0.00129027,     6
     -0.27589787,     -0.00180638,     7
     -0.36609308,     -0.00244856,     8
     -0.46675857,     -0.00323144,     9
      0.00602455,      0.00003845,    10
      0.01512194,      0.00009652,    11
      0.03086721,      0.00019705,    12
      0.05526326,      0.00035304,    13
      0.09031826,      0.00057792,    14
      0.13791345,      0.00088560,    15
      0.19954936,      0.00129027,    16
      0.27589787,      0.00180638,    17
      0.36609308,      0.00244856,    18
      0.46675857,      0.00323144,    19
     -0.00000000,     -0.00000000,    20
     -0.00000000,     -0.00000000,    21
     -0.00000000,     -0.00000000,    22
     -0.00000000,     -0.00000000,    23
 

      0.14012854,      0.04281474,  4776
      0.11162173,      0.04384989,  4777
      0.08499654,      0.04403754,  4778
      0.06058782,      0.04324884,  4779
      0.03969418,      0.04139969,  4780
     -0.00000000,     -0.00000000,  4781
     -0.00000000,     -0.00000000,  4782
     -0.00000000,     -0.00000000,  4783
     -0.00000000,     -0.00000000,  4784
     -0.00000000,     -0.00000000,  4785
     -0.00000000,     -0.00000000,  4786
     -0.00000000,     -0.00000000,  4787
     -0.00000000,     -0.00000000,  4788
     -0.00000000,     -0.00000000,  4789
     -0.00000000,     -0.00000000,  4790
     -0.00000000,     -0.00000000,  4791
     -0.00000000,     -0.00000000,  4792
     -0.00000000,     -0.00000000,  4793
     -0.00000000,     -0.00000000,  4794
     -0.00000000,     -0.00000000,  4795
     -0.00000000,     -0.00000000,  4796
     -0.00000000,     -0.00000000,  4797
     -0.00000000,     -0.00000000,  4798
     -0.00000000,     -0.00000000,  4799
     -0.00000000

     -0.00825721,     -0.01352599,  8380
      0.00825721,      0.01352599,  8381
     -0.00825721,     -0.01352599,  8382
      0.00825721,      0.01352599,  8383
     -0.00825721,     -0.01352599,  8384
      0.00825721,      0.01352599,  8385
     -0.00825721,     -0.01352599,  8386
      0.00825721,      0.01352599,  8387
     -0.00825721,     -0.01352599,  8388
      0.00825721,      0.01352599,  8389
     -0.02037841,     -0.03338150,  8390
      0.02037841,      0.03338150,  8391
     -0.02037841,     -0.03338150,  8392
      0.02037841,      0.03338150,  8393
     -0.02037841,     -0.03338150,  8394
      0.02037841,      0.03338150,  8395
     -0.02037841,     -0.03338150,  8396
      0.02037841,      0.03338150,  8397
     -0.01542239,     -0.02526313,  8398
      0.01542239,      0.02526313,  8399
     -0.01542239,     -0.02526313,  8400
      0.01542239,      0.02526313,  8401
     -0.01542239,     -0.02526313,  8402
      0.01542239,      0.02526313,  8403
     -0.01542239

     -0.13976243,     -0.04025670, 12308
     -0.13032072,     -0.03999648, 12309
     -0.00137183,     -0.01453714, 12310
     -0.00053495,     -0.01149358, 12311
     -0.00017482,     -0.00866744, 12312
     -0.00004616,     -0.00617282, 12313
     -0.00000940,     -0.00409940, 12314
     -0.00000139,     -0.00249672, 12315
     -0.00000014,     -0.00136360, 12316
     -0.00000001,     -0.00064730, 12317
     -0.18999050,     -0.03805788, 12318
     -0.18604980,     -0.03754438, 12319
     -0.18031244,     -0.03692170, 12320
     -0.17250579,     -0.03617209, 12321
     -0.00285858,     -0.00803299, 12322
     -0.00116599,     -0.00582598, 12323
     -0.00039513,     -0.00395049, 12324
     -0.00010643,     -0.00245703, 12325
     -0.00002143,     -0.00136097, 12326
     -0.00000293,     -0.00063749, 12327
     -0.00000023,     -0.00022468, 12328
     -0.00000001,     -0.00003589, 12329
     -0.13978206,     -0.04184482, 12330
     -0.13133248,     -0.04192797, 12331
     -0.12210014

     -0.12784688,     -0.03873950, 16439
     -0.11648965,     -0.03813198, 16440
     -0.10428402,     -0.03735771, 16441
     -0.09143595,     -0.03639025, 16442
     -0.07820205,     -0.03520374, 16443
     -0.06490475,     -0.03377467, 16444
     -0.05194450,     -0.03208435, 16445
     -0.03978825,     -0.03012177, 16446
     -0.02891846,     -0.02788698, 16447
     -0.01974777,     -0.02539461, 16448
     -0.01252717,     -0.02267713, 16449
     -0.00728480,     -0.01978729, 16450
     -0.14994602,     -0.03642978, 16451
     -0.13803663,     -0.03553638, 16452
     -0.12469647,     -0.03447083, 16453
     -0.11028649,     -0.03321412, 16454
     -0.09523742,     -0.03174972, 16455
     -0.08000296,     -0.03006545, 16456
     -0.06503943,     -0.02815574, 16457
     -0.05081573,     -0.02602411, 16458
     -0.03782317,     -0.02368579, 16459
     -0.02654186,     -0.02117006, 16460
     -0.01735128,     -0.01852190, 16461
     -0.01042384,     -0.01580247, 16462
     -0.12784688

In [35]:
dm = method.init_guess_by_1e()
coords = numpy.asarray(method.grids.coords)
ni = dft.numint.NumInt()
ao_value = pyscf.dft.numint.eval_ao(mol,coords,deriv=0)
rho_value_a = pyscf.dft.numint.eval_rho(mol,ao_value,dm[0],xctype='LDA')
rho_value_b = pyscf.dft.numint.eval_rho(mol,ao_value,dm[1],xctype='LDA')
print(rho_value.shape)
for x in range(100):
    print("%16.8f,%16.8f" %(rho_value_a[x],rho_value_b[x]))

Initial guess from hcore.
  HOMO = -0.611211671131453  LUMO = -0.315915792325708
(17950,)
      0.37883036,      0.22537147
      0.37882760,      0.22561417
      0.37879265,      0.22601880
      0.37866282,      0.22660739
      0.37831442,      0.22737199
      0.37753192,      0.22825739
      0.37597360,      0.22914098
      0.37314096,      0.22981257
      0.36836651,      0.22996116
      0.36084569,      0.22918145
      0.37881436,      0.22503993
      0.37878744,      0.22478195
      0.37871061,      0.22431979
      0.37851567,      0.22356408
      0.37807284,      0.22239232
      0.37715946,      0.22063418
      0.37542454,      0.21805571
      0.37235604,      0.21434668
      0.36726532,      0.20911916
      0.35931380,      0.20193051
      0.37882226,      0.22520556
      0.37880686,      0.22519717
      0.37874886,      0.22516561
      0.37858037,      0.22507389
      0.37816985,      0.22485041
      0.37728984,      0.22437129
      0.37558056,      0.2

In [17]:
# store the grids to an external file
print(len(method.grids.weights))
ff = open("grids-3",'w')
for i in range(len(method.grids.weights)):
    x,y,z = method.grids.coords[i]
    w = method.grids.weights[i]
    ff.write("%16.8e,%16.8e,%16.8e,%16.8e\n" %(x,y,z,w))
ff.close()

17950


In [14]:
TimeStart = time()
mol = gto.Mole(                                                                                               
        atom='''
         N       -2.1988391019      1.8973746268      0.0000000000                 
         H       -1.1788391019      1.8973746268      0.0000000000                 
         H       -2.5388353987      1.0925460144     -0.5263586446                 
         H       -2.5388400276      2.7556271745     -0.4338224694 ''',
        basis='def2-tzvp',verbose=0
    ).build()
method = dft.RKS(mol)
print('Default DFT(LDA).  E = %.12f' % method.kernel())

# See pyscf/dft/radi.py for more radial grid schemes
#grids.radi_method = dft.gauss_chebeshev
#grids.radi_method = dft.delley
method = dft.RKS(mol)
method.grids.radi_method = dft.mura_knowles
print('Changed radial grids for DFT.  E = %.12f' % method.kernel())


# See pyscf/dft/gen_grid.py for detail of the grid weight scheme
#method.grids.becke_scheme = dft.original_becke
# Stratmann-Scuseria weight scheme
method = dft.RKS(mol)
method.grids.becke_scheme = dft.stratmann
print('Changed grid partition funciton.  E = %.12f' % method.kernel())

# Grids level 0 - 9.  Big number indicates dense grids. Default is 3
method = dft.RKS(mol)
method.grids.level = 4
print('Dense grids.  E = %.12f' % method.kernel())

# Specify mesh grid for certain atom
method = dft.RKS(mol)
method.grids.atom_grid = {'O': (100, 770)}
print('Dense grids for O atom.  E = %.12f' % method.kernel())

# Specify mesh grid for all atoms
method = dft.RKS(mol)
method.grids.atom_grid = (100, 770)
print('Dense grids for all atoms.  E = %.12f' % method.kernel())


# Disable pruning grids near core region
#grids.prune = dft.sg1_prune
method = dft.RKS(mol)
method.grids.prune = None
print('Changed grid partition funciton.  E = %.12f' % method.kernel())

print("Total job time: %10.2f(wall)" %(time()-TimeStart))

Default DFT(LDA).  E = -56.100667658113
Changed radial grids for DFT.  E = -56.100667683944
Changed grid partition funciton.  E = -56.100667820637
Dense grids.  E = -56.100667719163
Dense grids for O atom.  E = -56.100667658113
Dense grids for all atoms.  E = -56.100667729458
Changed grid partition funciton.  E = -56.100667639101
Total job time:       2.85(wall)


In [26]:
grids = dft.gen_grid.Grids(mol)
grids.atomic_radii
#help(dft.gen_grid.Grids)

array([3.77945036, 0.66140414, 2.64561657, 2.74010288, 1.98421243,
       1.60626721, 1.32280829, 1.22832198, 1.13383567, 0.94486306,
       2.83458919, 3.40150702, 2.83458919, 2.36215766, 2.07869874,
       1.88972612, 1.88972612, 1.88972612, 3.40150702, 4.15739747,
       3.40150702, 3.0235618 , 2.64561657, 2.55113027, 2.64561657,
       2.64561657, 2.64561657, 2.55113027, 2.55113027, 2.55113027,
       2.55113027, 2.45664396, 2.36215766, 2.17318504, 2.17318504,
       2.17318504, 3.59047964, 4.44085639, 3.77945225, 3.40150702,
       2.92907549, 2.74010288, 2.74010288, 2.55113027, 2.45664396,
       2.55113027, 2.64561657, 3.0235618 , 2.92907549, 2.92907549,
       2.74010288, 2.74010288, 2.64561657, 2.64561657, 3.96842486,
       4.91328792, 4.06291117, 3.68496594, 3.49599333, 3.49599333,
       3.49599333, 3.49599333, 3.49599333, 3.49599333, 3.40150702,
       3.30702072, 3.30702072, 3.30702072, 3.30702072, 3.30702072,
       3.30702072, 3.30702072, 2.92907549, 2.74010288, 2.55113

In [15]:
# get density matrix from '1e'
print("Now for density matrix")
dm = method.init_guess_by_1e()
for x in dm:
    print("%10.6f"*10 %tuple([y for y in x]))
print("Now for overlap matrix")
om = method.get_ovlp()
for x in om:
    print("%10.6f"*10 %tuple([y for y in x]))

Now for density matrix
Initial guess from hcore.
  HOMO = -1.26716325458686  LUMO = -0.611211671131453
  0.500612  0.060052  0.047979  0.000000 -0.000000  0.500612  0.060052 -0.047979 -0.000000  0.000000
  0.060052  0.007204  0.005755  0.000000 -0.000000  0.060052  0.007204 -0.005755 -0.000000  0.000000
  0.047979  0.005755  0.004598  0.000000 -0.000000  0.047979  0.005755 -0.004598 -0.000000  0.000000
  0.000000  0.000000  0.000000  0.000000 -0.000000  0.000000  0.000000 -0.000000 -0.000000  0.000000
 -0.000000 -0.000000 -0.000000 -0.000000  0.000000 -0.000000 -0.000000  0.000000  0.000000 -0.000000
  0.500612  0.060052  0.047979  0.000000 -0.000000  0.500612  0.060052 -0.047979 -0.000000  0.000000
  0.060052  0.007204  0.005755  0.000000 -0.000000  0.060052  0.007204 -0.005755 -0.000000  0.000000
 -0.047979 -0.005755 -0.004598 -0.000000  0.000000 -0.047979 -0.005755  0.004598  0.000000 -0.000000
 -0.000000 -0.000000 -0.000000 -0.000000  0.000000 -0.000000 -0.000000  0.000000  0.00000

In [17]:
dm = method.init_guess_by_1e()
ao_value = dft.numint.eval_ao(mol,method.grids.coords,deriv=0)
TimeStart = time()
rho_a=dft.numint.eval_rho(mol,ao_value,dm[0],xctype='LDA')
rho_b=dft.numint.eval_rho(mol,ao_value,dm[1],xctype='LDA')
print(len(rho_a),len(rho_b),len(ao_value[0]))
print("Total job time: %10.2f(wall)" %(time()-TimeStart))

Initial guess from hcore.
  alpha nocc = 2  HOMO = -0.616036367261958  LUMO = -0.380396958782236
  beta  nocc = 1  HOMO = -1.27108281571849  LUMO = -0.616036367261958

WARN: system HOMO -0.616036367261958 >= system LUMO -0.616036367261958

18392 18392 60
Total job time:       0.01(wall)


In [14]:
help(method.get_ovlp)

Help on method get_ovlp in module pyscf.scf.hf:

get_ovlp(mol=None) method of pyscf.dft.rks.RKS instance



In [40]:
"16.8f"*10

'16.8f16.8f16.8f16.8f16.8f16.8f16.8f16.8f16.8f16.8f'